In [1]:
import numpy as np
import torch
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

In [2]:
X, y = load_breast_cancer(return_X_y=True)

In [3]:
X = StandardScaler().fit_transform(X)
y = y[..., np.newaxis]

In [4]:
# X = X[:2]
# y = y[:2]

In [5]:

w1 = torch.zeros(30, 1)
# w2 = torch.ones(2, 1)
b1 = torch.zeros(1)
# b2 = torch.zeros(1)
X_t = torch.from_numpy(X).float()
y_t = torch.from_numpy(y).float()

In [6]:
%%time
for i in range(1000):
    
    w1.requires_grad = True
    b1.requires_grad = True
#     w2.requires_grad = True
#     b2.requires_grad = True
    
#     a1 = X_t @ w1 + b1
#     print(a1.shape)
#     print((a1 @ w2).shape)
    y__t = X_t @ w1 + b1
    
    
#     y__t.requires_grad = True
#     print(y__t.shape, y_t.shape)
    
    assert y__t.shape == y_t.shape
#     print(y__t, y_t)
#     l = torch.nn.functional.binary_cross_entropy_with_logits(y__t, y_t)
    y__t = torch.sigmoid(y__t)
    l = -torch.sum(y_t * torch.log(y__t) + (1 - y_t) * torch.log(1 - y__t)) / len(y_t)
    l.backward()
#     print(w2.grad)
#     print(b2.grad)
#     print(w1.grad)
#     print(b1.grad)
    if i % 100 == 0:
        print(l)
    w1 = w1.data - 0.01 * w1.grad
#     w2 = w2.data - 0.01 * w2.grad
    
    b1 = b1.data - 0.01 * b1.grad
#     b2 = b2.data - 0.01 * b2.grad



tensor(0.6931, grad_fn=<DivBackward0>)
tensor(0.2489, grad_fn=<DivBackward0>)
tensor(0.1865, grad_fn=<DivBackward0>)
tensor(0.1583, grad_fn=<DivBackward0>)
tensor(0.1416, grad_fn=<DivBackward0>)
tensor(0.1303, grad_fn=<DivBackward0>)
tensor(0.1221, grad_fn=<DivBackward0>)
tensor(0.1158, grad_fn=<DivBackward0>)
tensor(0.1107, grad_fn=<DivBackward0>)
tensor(0.1066, grad_fn=<DivBackward0>)
CPU times: user 3.37 s, sys: 195 ms, total: 3.56 s
Wall time: 548 ms


In [7]:
class Linear:
    def __init__(self, in_dim, out_dim):
        self.w = np.zeros([in_dim, out_dim])
#         self.w = np.random.randn(in_dim, out_dim)
        self.b = np.zeros([1, out_dim])
        self.dw = None
        self.db = None
        self.in_dim = self.w.shape[0]
        self.out_dim = self.w.shape[1]

        
    def forward(self, x):
        self.x = x
        return np.matmul(x, self.w) + self.b
    
    def backward(self, d):
        self.db = np.mean(d, axis=0)
        assert self.db.shape == self.b.shape, (d.shape, self.db.shape, self.b.shape)
        
        J = np.zeros([self.x.shape[0], self.out_dim, np.prod(self.w.shape)])
        j = 0
        for i in range(self.out_dim):
            J[:, i: i + 1, j: j + self.in_dim] = self.x[:, np.newaxis, :]
            j += self.in_dim
        
        dw = d @ J
        
        dw = np.reshape(np.mean(dw, axis=0), self.w.shape, order='F')
        
        self.dw = dw
        
        d = d @ np.repeat(self.w.T[np.newaxis, ...], d.shape[0], axis=0)
        
        return d
        
    def step(self, lr):
        self.w = self.w - lr * self.dw
        self.b = self.b - lr * self.db

In [8]:
class Sigmoid:
    def __init__(self):
        self.a = None
    def forward(self, x):
        self.a = 1 / (1 + np.exp(-x))
        return self.a
    def backward(self, d):
        J = (self.a * (1 - self.a))[:, np.newaxis, ...]
        assert J.shape == d.shape
        return d * J

In [9]:
class BinaryCrossEntropy:
    def forward(self, y_, y):
        assert y.shape == y_.shape, (y.shape, y_.shape)
        l = -np.sum(y * np.log(y_) + (1 - y) * np.log(1 - y_))
        l /= len(y)
        return y_, l
    
    def backward(self, y_, y):
        assert y_.shape == y.shape
        d = - y / (y_) + (1 - y) / (1 - y_)
        d = d[:, np.newaxis, ...]
        print(d.shape)
        raise
        return d

In [10]:
class LogisticRegression:
    def __init__(self):
        self.linear = Linear(30, 1)
        self.sigmoid = Sigmoid()
        self.loss = BinaryCrossEntropy()
        
    def forward(self, x, y):
        x = self.linear.forward(x)
        x = self.sigmoid.forward(x)
        loss = self.loss.forward(x, y)
        return loss
    
    def backward(self, x, y_, y):
        d = self.loss.backward(y_, y)
        d = self.sigmoid.backward(d)
        self.linear.backward(d)
    
    def step(self, lr):
        self.linear.step(lr)
        

In [11]:
logreg = LogisticRegression()

In [12]:
%%time
for i in range(1000):
    y_, loss = logreg.forward(X, y)
    if i % 100 == 0:
        print(loss, 'loss')
    logreg.backward(X, y_, y)
    logreg.step(0.01)


0.6931471805599453 loss
(569, 1, 1)


RuntimeError: No active exception to reraise